In [6]:
import MySQLdb
import pandas
import itertools
import numpy as np
import bm25fe
import pickle

In [7]:
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur = conn.cursor()

sql = '''SELECT bug_id, short_desc, long_desc
FROM bugs_cpdplatform'''

# bugs = pandas.io.sql.read_sql(sql, conn).set_index(['bug_id'])
bugs = pandas.io.sql.read_sql(sql, conn)

In [8]:
with open('train100.txt', 'rb') as f:
    train = pickle.load(f)

bugset = []
for item in train:
    bugset.append(item['query'])
    bugset.append(item['rel'])
    bugset += item['irrel']

bugset = set(bugset)
# bugset = set(itertools.chain(*train))

criterion = bugs['bug_id'].map(lambda x: x in bugset)
bugs_train = bugs[criterion]

In [9]:
from gensim import corpora
from gensim import matutils

# dictionary = corpora.Dictionary(list(bugs_train['text']))
# dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n = 100000)
dictionary = corpora.Dictionary.load_from_text('dictionary.txt')
num_terms = len(dictionary)
print num_terms
bugs_train.loc[:,'short_desc'] = bugs_train['short_desc'].map(lambda x: matutils.corpus2dense([dictionary.doc2bow(x.split())], num_terms, 1)[:,0])
bugs_train.loc[:,'long_desc'] = bugs_train['long_desc'].map(lambda x: matutils.corpus2dense([dictionary.doc2bow(x.split())], num_terms, 1)[:,0])

8647


/root/.virtualenvs/dup_bug_retrieval/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
idf = np.log(bugs_train.shape[0]/np.array(dictionary.dfs.values()))
avgfl = np.array([np.array(list(bugs_train['short_desc'])).sum(1).mean(), np.array(list(bugs_train['long_desc'])).sum(1).mean()])

bugs_train = bugs_train.set_index(['bug_id'])

In [ ]:
def cost(bm, bugs, train, idf, avgfl):
    cost = 0
    for item in train:
        for bug in item['irrel']:
            differ = (bm.score(idf, avgfl, [bugs.loc[bug,'short_desc'], bugs.loc[bug,'long_desc']], [bugs.loc[item['query'],'short_desc'], bugs.loc[item['query'],'long_desc']])-
                      bm.score(idf, avgfl, [bugs.loc[item['rel'],'short_desc'], bugs.loc[item['rel'],'long_desc']], [bugs.loc[item['query'],'short_desc'], bugs.loc[item['query'],'long_desc']]))
            cost += np.log(1+np.exp(differ))
    return cost

In [ ]:
params = np.array([1.2, 0.75, 0.75, 2, 1, 1.2, 0.75, 0.75, 2, 1])
step = 1e-6
beta = 0.8
for i in xrange(24):
    for item in train:
        bm = bm25fe.bm25fe(K1=params[0], d_B=(params[1], params[2]), d_W = (params[3], params[4]), K3=params[5], q_B=(params[6], params[7]), q_W=(params[8], params[9]))
        der = np.zeros(params.size)
        for bug in item['irrel']:
            differ = (bm.score(idf, avgfl, [bugs_train.loc[bug,'short_desc'], bugs_train.loc[bug,'long_desc']], [bugs_train.loc[item['query'],'short_desc'], bugs_train.loc[item['query'],'long_desc']])-
                      bm.score(idf, avgfl, [bugs_train.loc[item['rel'],'short_desc'], bugs_train.loc[item['rel'],'long_desc']], [bugs_train.loc[item['query'],'short_desc'], bugs_train.loc[item['query'],'long_desc']]))
            der += np.exp(differ)/(1+np.exp(differ))*(bm.derivative(idf, avgfl, [bugs_train.loc[bug,'short_desc'], bugs_train.loc[bug,'long_desc']],[bugs_train.loc[item['query'],'short_desc'], bugs_train.loc[item['query'],'long_desc']])-
                                                      bm.derivative(idf, avgfl, [bugs_train.loc[item['rel'],'short_desc'], bugs_train.loc[item['rel'],'long_desc']],[bugs_train.loc[item['query'],'short_desc'], bugs_train.loc[item['query'],'long_desc']]))
        while 1:
            inner = params - der*step
            bm_inner = bm25fe.bm25fe(K1=inner[0], d_B=(inner[1], inner[2]), d_W = (inner[3], inner[4]), K3=inner[5], q_B=(inner[6], inner[7]), q_W=(inner[8], inner[9]))
            if cost(bm_inner, bugs_train, train, idf, avgfl) < cost(bm, bugs_train, train, idf, avgfl):
                break
            step = beta*step
        params = inner
            
print params

In [118]:
# print type(vectors[:n])
# print vectors[0]
# print len(vectorizer.vocabulary_)
# print bug_feature.iloc[0,1]
# print list(vectors_sd)[0]
# print vectors_sd.sum(1).mean()

In [90]:
# print train
# print str(915183) in bugset
# print bugset
# print len(bugset)
# print bugs_train.shape

False
3498
(3498, 3)
